In [5]:
import pandas as pd
import numpy as np
from itertools import combinations
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en.examples import sentences 

In [8]:
nlp = spacy.load("en_core_web_sm")

In [92]:
df = pd.read_csv('/Users/dougl/Desktop/Categories/Definitions_Ext/final_def_data_FULL.csv', encoding='cp1252', na_values=['nan'])
#df.head()

In [12]:
pd.set_option('display.max_colwidth', 0)

In [80]:
clu_ids = set(df["instance_id"])
clu_wordembedding_concentration_data = []
clu_wordembedding_concentration_dists = {}
clu_wordembedding_concreteness_dists = {}

In [81]:
for clu_id in clu_ids:
    clu_id_data = df[(df["instance_id"]==clu_id)]
    categories = set(clu_id_data['label'])
    currN = list(set(clu_id_data['N']))[0]
    
    for category in categories:
        cat_data = clu_id_data[(clu_id_data["label"]==category)]
        clu_id_words = [str(x).split(" ") for x in list(cat_data["definition"])]
        combo_def = " ".join([" ".join(x) for x in clu_id_words])
        
        if not(clu_id_words): continue
        clu_doc = nlp(combo_def)
        doc_vec = clu_doc.vector
        similarities = []
        for word_idx, word in enumerate(clu_doc):
            wordstr = str(word)
            word_vec = clu_doc[word_idx].vector        
            if sum(word_vec) == 0.0: continue
            cos_sim = cosine_similarity([word_vec],[doc_vec])
            similarities.append(cos_sim)
        clu_wordembedding_concentration_data.append([currN,clu_id,category,np.mean(similarities)])    

In [93]:
#clu_wordembedding_concentration_data

In [89]:
df = pd.DataFrame(clu_wordembedding_concentration_data)

In [90]:
df.rename(columns={0: "N", 1: "instance_id", 2:"label",3:"similarity"})

,N,instance_id,label,similarity
0,8,392,bell,0.246265
1,8,392,frog,0.214682
2,8,392,crab,0.286321
3,8,393,x,0.228317
4,8,393,couch,0.219204
...,...,...,...,...
142,24,727,crab,0.237840
143,24,727,ball,0.264360
144,24,728,frog,0.233224
145,24,728,bunny,0.312912


In [91]:
df.to_csv('FULL_dis_DG.csv', index=True)